<a href="https://colab.research.google.com/github/AshhKetchup/Mini-P/blob/main/SLM_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
ds={}

In [31]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pickle
import re

# Setup model and tokenizer
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Ensure the model is on the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def run_chain_of_thought(selected_task):
    problem_statement = "I am a highly intelligent question answering bot and I answer questions from a human perspective."

    # Step 1: Get objects prompt
    objects_prompt = """Q: Which common objects in daily life can be used as a tool for humans to {}?
    Please list 20 most suitable objects. Objects should be different from each other."""

    # Step 2: Get rationales prompt
    rationales_prompt = """For each object listed above, please explain the rationales for why they afford the task of {}
    from the perspective of visual features."""

    # Step 3: Get visual features prompt
    features_prompt = """For each object and its rationales, please summarize the corresponding visual features in one sentence,
    with comma-separated values of features, where each feature is described briefly.

    For example:
    1. [Object name]:
    Rationales: [Rationale from previous response]
    Visual Features: [feature 1], [feature 2], [feature 3], ..."""



    # Step 1: Get objects
    messages_step1 = [
        {"role": "system", "content": problem_statement},
        {"role": "user", "content": objects_prompt.format(selected_task)}
    ]
    input_text_step1 = tokenizer.apply_chat_template(messages_step1, tokenize=False)
    # print("STEP 1: Get objects\n" )


    inputs_step1 = tokenizer.encode(input_text_step1, return_tensors="pt").to(device)
    outputs_step1 = model.generate(inputs_step1, max_new_tokens=1000, temperature=0.2, top_p=0.9, do_sample=True)
    response_step1 = tokenizer.decode(outputs_step1[0])
    # print("Response for Step 1:\n" + response_step1)


    # Step 2: Get rationales
    messages_step2 = [
        {"role": "system", "content": problem_statement},
        {"role": "user", "content": objects_prompt.format(selected_task)},
        {"role": "assistant", "content": response_step1},
        {"role": "user", "content": rationales_prompt.format(selected_task)}
    ]
    input_text_step2 = tokenizer.apply_chat_template(messages_step2, tokenize=False)
    # print("\nSTEP 2: Get rationales\n")

    inputs_step2 = tokenizer.encode(input_text_step2, return_tensors="pt").to(device)
    outputs_step2 = model.generate(inputs_step2, max_new_tokens=1500, temperature=0.2, top_p=0.9, do_sample=True)
    response_step2 = tokenizer.decode(outputs_step2[0])
    # print("Response for Step 2:\n" + response_step2)

    # Step 3: Get visual features
    messages_step3 = [
        {"role": "system", "content": problem_statement},
        {"role": "user", "content": objects_prompt.format(selected_task)},
        {"role": "assistant", "content": response_step1},
        {"role": "user", "content": rationales_prompt.format(selected_task)},
        {"role": "assistant", "content": response_step2},
        {"role": "user", "content": features_prompt}
    ]
    input_text_step3 = tokenizer.apply_chat_template(messages_step3, tokenize=False)
    # print("\nSTEP 3: Get visual features\n")

    inputs_step3 = tokenizer.encode(input_text_step3, return_tensors="pt").to(device)
    outputs_step3 = model.generate(inputs_step3, max_new_tokens=2000, temperature=0.2, top_p=0.9, do_sample=True)
    response_step3 = tokenizer.decode(outputs_step3[0])
    # print("Response for Step 3:\n" + response_step3)

    return response_step3

# List of tasks
tasks = [
    "step on", "sit comfortably on", "place flowers in",
    "get potatoes out of fire with", "water plant with",
    "get lemon out of tea with", "dig hole with",
    "open bottle of beer with", "open parcel with",
    "serve wine with", "pour sugar with",
    "smear butter with", "extinguish fire with",
    "pound carpet with"
]

# run_chain_of_thought(tasks[0]) works finally bitchhhh


from transformers import RobertaTokenizer, RobertaModel
import torch
r_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model1 = RobertaModel.from_pretrained('roberta-base')
model1.to(device)
def get_padded_embeddings(text, max_tokens=42):
    # Tokenize with device awareness
    inputs = r_tokenizer(
        text,
        return_tensors="pt",
        max_length=max_tokens,
        truncation=True,
        padding='max_length'
    ).to(device)  # Move inputs to correct device

    # Get model outputs
    with torch.no_grad():
        outputs = model1(**inputs)

    # Get embeddings
    word_embeddings = outputs.last_hidden_state.squeeze()
    sentence_embedding = word_embeddings.mean(dim=0)

    return {
        'text': text,
        'input_ids': inputs['input_ids'].squeeze().cpu().tolist(),  # Move to CPU for storage
        'attention_mask': inputs['attention_mask'].squeeze().cpu().tolist(),
        'word_embeddings': word_embeddings.cpu(),  # Move to CPU
        'sentence_embedding': sentence_embedding.cpu(),  # Move to CPU
        'total_tokens': max_tokens
    }

import re
for task in tasks[12:]:
  response_step3 = run_chain_of_thought(task)
  features = re.findall(r"Visual Features: (.+)", response_step3)
  features = features[1:6]
  embedding_results = [get_padded_embeddings(elem) for elem in features]

  # Create word embeddings tensor with requires_grad=True
  word_emb = [result["word_embeddings"] for result in embedding_results]
  word_emb_tensor = torch.stack(word_emb)
  word_emb_tensor.requires_grad = True

  # Create sentence embeddings tensor with requires_grad=True
  sentence_emb = [result["sentence_embedding"] for result in embedding_results]
  sentence_emb_tensor = torch.stack(sentence_emb)
  sentence_emb_tensor.requires_grad = True

  # Store in ds dictionary
  ds[task] = (word_emb_tensor, sentence_emb_tensor)

  print(f"DONE {task}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DONE extinguish fire with
DONE pound carpet with


In [34]:
with open('final.pkl', 'wb') as handle:
    pickle.dump(ds, handle, protocol=pickle.HIGHEST_PROTOCOL)

UnpicklingError: invalid load key, '\x16'.